## Como os deputados votam: Extração dos dados
### Um projeto de aprendizado não supervisado


Quando o assunto é política, todo mundo tem um pitaco para dar: seja nos valores que a política deve ter; seja no que cada politico deveria fazer para melhorar nosso país; seja nos reajustes necessários na legislação brasileira. 

Mas, quando passamos a falar e politica na prática, considerando os projetos de lei, votações na câmara e decretos governamentais ninguém entende muito bem. Se tratando de um país do tamanho do Brasil é entendível que as regras de governo sejam complexas, cheias de processos e burocracias. É parte do sistema democrático.

Uma parte importante desse processo são as votações na câmara dos deputados. É la que são aprovados projetos, leis ementas e outras coisas do direito que mudam diretamente ou indiretamente nossa vida enquanto cidadãos. O interessante é que câmara legislativa disponibiliza um volume imenso de dados sobre o que acontece nessas votações, nos permitindo e incentivando a explorar esses dados.

Sabendo que a política é atualmente um assunto muito importante para nós brasileiros, mas que não entendemos muito bem o comportamento dos parlamentares nas votações e existem dados que nos permitem explorar um pouco esse comportamento, surgiu a ideia de desenvolver esse projeto: um estudo sobre os dados das votações na câmara dos deputados.

Essa é a primeira parte de três. Aqui eu irei extrair os dados da plataforma da câmara legislativa e explicar um pouco como eu fiz para interagir com a plataforma. 


### Preparativos

Começamos efetuando alguns procedimentos gerais de extração dos dados. No meu caso eu sempre gosto de começar importando as bibliotecas que eu quase sempre uso e definindo os “header” s para a interação com a plataforma.



In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from bs4 import BeautifulSoup
from requests import get
import itertools


from tqdm.notebook import tqdm, trange
import time 

headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

Em seguida eu decidi definir uma função que basicamente extrai os dados da plataforma conforme a url. Aqui cabe dizer que cada categoria de dado que extrairemos aqui tem um conjunto de especificações e organização diferente, mas a API da câmara legislativa sempre entrega eles num mesmo formato de JSON, onde os dados, estão na chave 'dados'.


In [2]:
def get_data(url):
    response = get(url, headers=headers)
    try:
        ret = json.loads(response.text)
    except ValueError:
        ret = {'dados':''}  #para resolver um bug do JSON
    return ret

### Interação com a plataforma

Tendo em mãos essa função já podemos testar a extração dos dados de uma url do site da câmara legislativa. Para interagir com essas urls automatizadamente nos precisamos de algumas coisas:


- a url em si
- como ela varia em parâmetros como datas, numero de página e etc
- quais as limitações (máximo de elementos por página, maior intervalo possível)
- qual o número máximo de páginas que aquele conjunto de dados gera


Todos esses dados estão disponíveis na própria pagina da API da câmara, precisamente nesse link aqui https://dadosabertos.camara.leg.br/swagger/api.html. 


#### deputados

Para pegar a url de um determinado conjunto de dados, como, por exemplo, os dados dos deputados existentes na câmara, basta:

- Ir ao link da API
- clicar no "GET" ao lado do conjunto de dados desejado (no nosso caso deputados)
- clicar em "Try it out"
- mudar os parâmetros caso necessário (coloquei "pagina" como 1 apenas para entender onde que isso varia na url)
- mudar "response content type" para "application/json"
- clicar em "execute"


PRRIIIIIINNNTTTTTSSSSSS 


E assim o site gera uma url (aparece embaixo de "Request URL") sendo o que utilizaremos para interagir com os dados por aqui.


Para facilitar meu entendimento, dividi as urls em antes e depois do número de página,  e ir variando esse numero para gerar uma url para cada página, e em seguida interagir com cada uma dessas urls.

In [ ]:
url_1='https://dadosabertos.camara.leg.br/api/v2/deputados?pagina='
url_2='&itens=60&ordem=ASC&ordenarPor=nome'
urls=[]
for p in range(1, 10):
    urls.append(url_1+str(p)+url_2)

A extração dos dados é feita como mostra o código abaixo: 

In [ ]:
cols=['id', 'nome', 'siglaUf', 'siglaPartido', 'idLegislatura']
deputados=pd.DataFrame(columns=cols)
for url in urls:
    data =get_data(url)
    for d in data['dados']:
        data_point={}
        for c in cols:
            data_point[c]=d[c]
        deputados=deputados.append(data_point, ignore_index=True)

Aqui separei o nome das colunas que quero em uma lista (cols) e ir puxando do dos dados cada um desses elementos, gerando uma linha da nossa futura tabela de dados. Essa linha é "apendada" no final da tabela, e no final desse loop tenho o dataset dos deputados. Por fim eu salvei esses dados no meu PC. Isso será algo muito importante ao longo do projeto, pois os dados que estamos extraindo tomam uma quantidade alta de tempo e não podem ser armazenados só na memória RAM tendo o risco de perde-los sob qualquer imprevisto.

In [ ]:
import os
path = 'E:\prog\PYMLAIDS\Camara dos deputados\data'

output_file = os.path.join(path,'deputados.csv')
deputados.to_csv(output_file, index=False)

Feito isso agora generalizaremos o 'for' que utilizamos para interagir em cada página usando uma função que faz a mesma coisa. Ela toma como parâmetro as 2 partes da url, as colunas e o número de páginas máximo, e extrai os dados da mesma forma que na função anterior. Nessa função adicionei um tqdm, sendo basicamente uma função que cria uma barra de progresso para o interador(nesse caso as url). Assim tenho uma noção de como está o andamento do processamento dos dados e se tiver algum bug ou erro no meio do caminho eu consigo ver qual url que está dando problema.

In [ ]:
def get_data_pages(url_1, url_2, cols, n_paginas=2):
    urls=[]
    for p in range(1,n_paginas+1):
        urls.append(url_1+str(p)+url_2)
    ret= pd.DataFrame(columns=cols)
    for url in tqdm(urls, leave=False):
        data =get_data(url)
        for d in data['dados']:
            data_point={}
            for c in cols:
                data_point[c]=d[c]
            ret=ret.append(data_point, ignore_index=True)
    return ret

#### votações

O próximo conjunto de dados que extrairemos são as votações. O que queremos é basicamente uma tabela com o identificador de cada votação, a data, o órgão que realizou e se foi aprovada ou não. O esquema é o mesmo do conjunto de dados anterior, exceto que agora precisamos especificar a data de início e de término do conjunto de dados. Primeiro testo com dados dos 2 primeiros meses de 2020.

In [ ]:
cols_votacoes=['id', 'data', 'siglaOrgao', 'aprovacao']
url_1_votacoes='https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2020-01-01&dataFim=2020-02-28&pagina='
url_2_votacoes= '&ordem=DESC&ordenarPor=dataHoraRegistro'
n_pag_votacoes=10
votacoes=get_data_pages(url_1_votacoes, url_2_votacoes, cols_votacoes, n_pag_votacoes)

Sabendo que a extração dos dados das votações esta funcionando bem hora de generalizar para vários intervalos de tempo entre janeiro de 2019 e hoje. Os intervalos de datas não podem ser grandes demais, pois quando isso acontece o site da câmara retorna erro por demorar demais para processar esses dados. Os intervalos também não podem ser muito pequenos, porque a extração de cada página da câmara dos deputados ficaria muito lenta e o código demoraria horrores para rodar. Usei 60 dias nesse caso como intervalo, escolhido de forma bem arbitraria.

A API da câmara dos deputados também tem uma peculiaridade relacionada às datas: ela não consegue retornar dados de dois anos diferentes na mesma url. Isso é facilmente resolvível com alguns 'if' s no meio do for. O site também não aceita pedidos com datas além de hoje, por isso é necessário limitar nosso intervalo em relação a hoje.

In [ ]:
from datetime import date, timedelta
date_0= date(2019, 1, 1)

date_i= date_0
date_f= date_0+timedelta(days = 60)

dates=[]
for i in range (1000):
    if date_f> date.today():
        dates.append([date_i, date.today()])
        break
    if date_f.year != date_i.year:
        dates.append([date_i, date(date_i.year,12,31)])
        date_i=date(date_f.year,1,1)
        date_f=date_i+timedelta(days = 61)
    else:
        dates.append([date_i, date_f])
        date_i=date_f+timedelta(days = 1)
        date_f=date_f+timedelta(days = 62)

O bloco das datas fica do lado esquerdo do bloco do número de página, por isso é mais fácil de implementa-lo na url_1. Eu preferi criar uma lista com todas as primeiras partes da url das votações e em seguida interagir com elas em um for, assim:

In [ ]:
url_1s=[]
for d in dates:
    url_1s.append('https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio='+str(d[0])+'&dataFim='+str(d[1])+'&pagina=')

In [ ]:
cols_votacoes=['id', 'data', 'siglaOrgao', 'aprovacao']

url_2_votacoes= '&ordem=DESC&ordenarPor=dataHoraRegistro'
n_pag_votacoes=10

votacoes=pd.DataFrame(columns=cols_votacoes)

import multiprocessing
from joblib import Parallel, delayed


num_cores = multiprocessing.cpu_count()
inputs = tqdm(url_1s)

if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(get_data_pages)(i,url_2_votacoes, cols_votacoes, n_pag_votacoes) for i in inputs)

Aqui eu preferi ao invés de implementar um simples for utilizar um pouco de processamento paralelo, pois essa extração estava prometendo demorar mais de horas para rodar, e se no meio disso a internet caísse eu perderia todo o progresso. No processamento paralelo isso é contornado um pouco, pois o código é executado mais rapidamente.

Da mesma forma que no conjunto anterior eu decidi salvar para não perder as horas que esse código demorou para rodar. 

In [ ]:
votacoes=pd.concat(processed_list)
output_file = os.path.join(path,'votacoes.csv')
votacoes.to_csv(output_file, index=False)

#### Votos

Agora é só extrair os votos de cada deputado nessas votações. A interação com a url é bem simples, é basicamente mudar o 'id' da votação no modelo e extrair o que está na chave 'dados'  do objeto JSON. Nem todas as votações têm os dados dos votos de cada deputado, mas o código contorna esse problema bem-visto que o site simplesmente retorna uma string vazio para a chae 'dados' nesses casos. Nesse caso eu preferi não fazer de forma paralela, pois diferente do caso anterior as paginas interagem muito rápido e paralelizar o código não é muito uma vantagem. 


In [ ]:
votos={}

url_1_votos='https://dadosabertos.camara.leg.br/api/v2/votacoes/'
url_2_votos='/votos' 
urls=[]
for ID in tqdm( votacoes['id']):
    url =url_1_votos+str(ID)+url_2_votos
    data =get_data(url)
    if len(data['dados'])>0:
        votos[ID]={d["deputado_"]['id']:d["tipoVoto"] for d in data['dados']}

        

Aqui cada linha representa um deputado (o 'index' do dataset é o id do deputado) e cada coluna representa uma votação. Mais uma vez salvamos os dados em um arquivo CSV para honrar as horas de processamento que gastei com esse código.

In [ ]:
votos=pd.DataFrame(votos)
output_file = os.path.join(path,'votos.csv')
votos.to_csv(output_file, index=True, index_label='cod_deputado')

#### Orientações


Por fim, eu decidi extrair também os dados sobre as orientações dos partidos e blocos a respeito das votações. A ideia aqui é possivelmente usar esses dados para entender o quanto os deputados estão alinhados com os seus partidos e talvez até imputar os votos dos deputados que estiverem faltando com essas orientações do partido. É importante dizer que as orientações podem ser identificadas por 'B' (de um bloco) ou 'P' (de um partido), e para ter um dataset mais conciso escolhi usar como index uma combinação desses fatores. Então, as orientações de um partido 'PPT' teriam como index 'PPT'.  

In [ ]:
orientacoes = {}
url_1_orientacoes = 'https://dadosabertos.camara.leg.br/api/v2/votacoes/'
url_2_orientacoes = '/orientacoes'
urls = []
for ID in tqdm(votos.columns):
    url = url_1_orientacoes+str(ID)+url_2_orientacoes
    data = get_data(url)
    if len(data['dados']) > 0:
        if d["codTipoLideranca"] == None: 
            orientacoes[ID] = {'N'+str(
                d['siglaPartidoBloco']): d["orientacaoVoto"] for d in data['dados']}
        else:
            orientacoes[ID] = {str(d["codTipoLideranca"])+str(
                d['siglaPartidoBloco']): d["orientacaoVoto"] for d in data['dados']}

Salvando mais uma vez os dados e assim terminando a extração os dados:)

In [ ]:
orientacoes=pd.DataFrame(orientacoes)
output_file = os.path.join(path,'orientacoes.csv')
orientacoes.to_csv(output_file, index=True, index_label='cod_partido')

### Proximos passos

Apos extrair esses dados, eu os limpei, filtrei e combinei nesse artigo aqui: LINK_1. Com esses dados tratados eu re-classifiquei os deputados com base em seu comportamento nas votações, nesse artigo aqui: LINK_2.